In [2]:
from __future__ import print_function
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math

# to not display the warnings of tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [3]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [34]:
def get_forecasting(values):
    look_back = 5
    nepochs = 200
    batch_size = 32

    dataset = values
    for i in range(len(dataset)):
        dataset[i] = [dataset[i]]
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    train = np.array(dataset)
    trainX, trainY = create_dataset(train, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
    
    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(8, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(trainX, trainY, epochs=nepochs, batch_size=batch_size, verbose=0)
    
    l = trainX[len(trainX) - 1]
    s = [list(x) for x in l]
    prediction = []
    for it in range(10):
        t = np.array([s])
        pred = model.predict(t)
        predt = scaler.inverse_transform(pred)
        k = []
        for i in range(1, len(s)):
            k.append(s[i])
        k.append(list(pred[0]))
        s = k
        prediction.append(int(predt))
    return prediction

In [35]:
data = [40, 50, 80, 100, 84, 98, 130, 125,
                    140, 180, 202, 219, 231, 207, 230,
                    250, 277, 291, 302, 315, 300,
                    312, 348, 367, 380, 350, 261, 289,
                    320, 390, 430, 467, 419, 422, 489,
                    512, 560, 620, 650, 710, 730, 722, 780,
                    820, 892, 878, 902, 950, 961, 988, 946]

In [36]:
p = get_forecasting(data)
print (p)

[1040, 1070, 1111, 1156, 1198, 1248, 1288, 1330, 1370, 1409]
